In [38]:
import pandas as pd
import torch
from pgmpy.models import BayesianModel
from tqdm import tqdm
from pgmpy.inference import VariableElimination

In [40]:
train_features = pd.read_csv('train_features.csv')
train_targets_scored = pd.read_csv('train_targets_scored.csv')

In [72]:
#Non encoded labels
cols = list(train_features.columns)
cols.append('target')
train_df = pd.DataFrame(columns=cols)

for i in tqdm(range(len(train_targets_scored))):
    for col in train_targets_scored.columns:
        if train_targets_scored.iloc[i][col] == 1:
            new_row = dict(train_features.iloc[i])
            new_row['target'] = col
            #add instance to train_df
            train_df = train_df.append(new_row, ignore_index=True)
            

100%|██████████████████████████████████████████████████████████████████████████| 23838/23838 [3:56:37<00:00,  1.68it/s]


In [75]:
train_df

,sig_id,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,...,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99,target
0,id_000644bb2,trt_cp,24,D1,1.0620,0.5577,-0.2479,-0.6208,-0.1944,-1.0120,...,0.2584,0.8076,0.5523,-0.1912,0.6584,-0.3981,0.2139,0.3801,0.4176,gsk_inhibitor
1,id_000a6266a,trt_cp,48,D1,0.6280,0.5817,1.5540,-0.0764,-0.0323,1.2390,...,-0.6297,0.6103,0.0223,-1.3240,-0.3174,-0.6417,-0.2187,-1.4080,0.6931,bcr-abl_inhibitor
2,id_000a6266a,trt_cp,48,D1,0.6280,0.5817,1.5540,-0.0764,-0.0323,1.2390,...,-0.6297,0.6103,0.0223,-1.3240,-0.3174,-0.6417,-0.2187,-1.4080,0.6931,kit_inhibitor
3,id_000a6266a,trt_cp,48,D1,0.6280,0.5817,1.5540,-0.0764,-0.0323,1.2390,...,-0.6297,0.6103,0.0223,-1.3240,-0.3174,-0.6417,-0.2187,-1.4080,0.6931,pdgfr_inhibitor
4,id_001626bd3,trt_cp,72,D2,-0.3254,-0.4009,0.9700,0.6919,1.4180,-0.8244,...,0.0048,0.6670,1.0690,0.5523,-0.3031,0.1094,0.2885,-0.3786,0.7125,calcium_channel_blocker
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16839,id_fff7e6992,trt_cp,48,D1,0.1420,-0.3696,-0.0093,-0.2495,-0.0175,0.0959,...,0.5230,0.1589,0.7715,0.5497,0.6929,0.3738,0.8636,0.4494,0.1218,anti-inflammatory
16840,id_fff8c2444,trt_cp,72,D1,0.1608,-1.0500,0.2551,-0.2239,-0.2431,0.4256,...,0.3538,0.0558,0.3377,-0.4753,-0.2504,-0.7415,0.8413,-0.4259,0.2434,progesterone_receptor_agonist
16841,id_fffb1ceed,trt_cp,24,D2,0.1394,-0.0636,-0.1112,-0.5080,-0.4713,0.7201,...,0.0262,-0.8121,0.3434,0.5372,-0.3246,0.0631,0.9171,0.5258,0.4680,serotonin_receptor_antagonist
16842,id_fffb70c0c,trt_cp,24,D2,-1.3260,0.3478,-0.3743,0.9905,-0.7178,0.6621,...,0.4426,0.0423,-0.3195,-0.8086,-0.9798,-0.2084,-0.1224,-0.2715,0.3689,ppar_receptor_agonist


In [74]:
train_df.to_csv('train_features_decoded.csv')

In [ ]:
#convert to binary high low for BN

for col in tqdm(list(train_df.columns)[4:-1]):
    stddev = np.std(train_df[col])
    mean = np.mean(train_df[col])
    for i in range(len(train_df)):
        if train_df[col].iloc[i] > 0:
            train_df[col].iloc[i] = 1
        else: 
            train_df[col].iloc[i] = 0
            
train_df[list(train_df.columns)[4:-1]] = train_df[list(train_df.columns)[4:-1]].astype(int)

train_df.to_csv('train_features_binary.csv')

In [ ]:
#maybe use something like this in the future
'''for col in tqdm(list(train_df.columns)[4:-1]):
    stddev = np.std(train_df[col])
    mean = np.mean(train_df[col])
    for i in range(len(train_df)):
        if train_df[col].iloc[i] > mean + stddev:
            train_df[col].iloc[i] = 2 #High
        elif train_df[col].iloc[i] < mean + stddev and train_df[col].iloc[i] > mean - stddev:
            train_df[col].iloc[i] = 1 #medium
        if train_df[col].iloc[i] < mean + stddev:
            train_df[col].iloc[i] = 0 #low'''